## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hasaki,JP,35.7333,140.8333,66.60,81,100,5.41,overcast clouds
1,1,Carnarvon,AU,-24.8667,113.6333,60.87,82,98,5.75,overcast clouds
2,2,Hamilton,US,39.1834,-84.5333,84.78,53,20,1.99,few clouds
3,3,Faanui,PF,-16.4833,-151.7500,80.83,73,99,8.50,overcast clouds
4,4,Hermanus,ZA,-34.4187,19.2345,53.80,59,0,4.38,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hamilton,US,39.1834,-84.5333,84.78,53,20,1.99,few clouds
3,3,Faanui,PF,-16.4833,-151.7500,80.83,73,99,8.50,overcast clouds
9,9,Jamestown,US,42.0970,-79.2353,83.08,44,1,16.11,clear sky
11,11,Paraiso,CR,9.8383,-83.8656,76.96,70,87,3.29,overcast clouds
13,13,Sinnamary,GF,5.3833,-52.9500,82.29,77,58,11.27,heavy intensity rain
16,16,Albany,US,42.6001,-73.9662,86.61,55,16,1.99,few clouds
21,21,Cidreira,BR,-30.1811,-50.2056,77.49,58,0,5.97,clear sky
22,22,Broome,US,42.2506,-75.8330,86.02,49,1,3.00,clear sky
24,24,Gizo,SB,-8.1030,156.8419,81.03,77,99,5.41,overcast clouds
27,27,Ano Mera,GR,37.4500,25.4000,75.58,53,0,18.41,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                272
City                   272
Country                272
Lat                    272
Lng                    272
Max Temp               272
Humidity               272
Cloudiness             272
Wind Speed             272
Current Description    272
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# not needed, no empty rows

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hamilton,US,84.78,few clouds,39.1834,-84.5333,
3,Faanui,PF,80.83,overcast clouds,-16.4833,-151.7500,
9,Jamestown,US,83.08,clear sky,42.0970,-79.2353,
11,Paraiso,CR,76.96,overcast clouds,9.8383,-83.8656,
13,Sinnamary,GF,82.29,heavy intensity rain,5.3833,-52.9500,
16,Albany,US,86.61,few clouds,42.6001,-73.9662,
21,Cidreira,BR,77.49,clear sky,-30.1811,-50.2056,
22,Broome,US,86.02,clear sky,42.2506,-75.8330,
24,Gizo,SB,81.03,overcast clouds,-8.1030,156.8419,
27,Ano Mera,GR,75.58,clear sky,37.4500,25.4000,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df[hotel_df['Hotel Name']!='']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hamilton,US,84.78,few clouds,39.1834,-84.5333,Six Acres Bed & Breakfast
3,Faanui,PF,80.83,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
9,Jamestown,US,83.08,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
11,Paraiso,CR,76.96,overcast clouds,9.8383,-83.8656,Rinconcito Verde
13,Sinnamary,GF,82.29,heavy intensity rain,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
21,Cidreira,BR,77.49,clear sky,-30.1811,-50.2056,Hotel Castelo
22,Broome,US,86.02,clear sky,42.2506,-75.8330,Chenango Valley State Park
24,Gizo,SB,81.03,overcast clouds,-8.1030,156.8419,Imagination Island
27,Ano Mera,GR,75.58,clear sky,37.4500,25.4000,Hotel La Résidence Mykonos
30,Gat,IL,76.82,clear sky,31.6100,34.7642,OlusHome


In [22]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hamilton,US,84.78,few clouds,39.1834,-84.5333,Six Acres Bed & Breakfast
3,Faanui,PF,80.83,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
9,Jamestown,US,83.08,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
11,Paraiso,CR,76.96,overcast clouds,9.8383,-83.8656,Rinconcito Verde
13,Sinnamary,GF,82.29,heavy intensity rain,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
21,Cidreira,BR,77.49,clear sky,-30.1811,-50.2056,Hotel Castelo
22,Broome,US,86.02,clear sky,42.2506,-75.8330,Chenango Valley State Park
24,Gizo,SB,81.03,overcast clouds,-8.1030,156.8419,Imagination Island
27,Ano Mera,GR,75.58,clear sky,37.4500,25.4000,Hotel La Résidence Mykonos
30,Gat,IL,76.82,clear sky,31.6100,34.7642,OlusHome


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))